In [1]:

!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
!unzip "/content/dataset.zip"

Archive:  /content/dataset.zip
   creating: dataset/
   creating: dataset/Test/
   creating: dataset/Test/Garbage/
  inflating: dataset/Test/Garbage/istockphoto-1175874693-612x612.jpg  
  inflating: dataset/Test/Garbage/istockphoto-1298617569-612x612.jpg  
  inflating: dataset/Test/Garbage/istockphoto-1313632033-612x612 (1).jpg  
  inflating: dataset/Test/Garbage/istockphoto-1472009122-612x612.jpg  
  inflating: dataset/Test/Garbage/istockphoto-1674278993-612x612.jpg  
  inflating: dataset/Test/Garbage/istockphoto-2027729122-612x612.jpg  
  inflating: dataset/Test/Garbage/overflow-scaled.jpg  
  inflating: dataset/Test/Garbage/overflowing-green-wheelie-bins_600x.jpg  
  inflating: dataset/Test/Garbage/overflowing-trash-bin-stockcake (1).jpg  
  inflating: dataset/Test/Garbage/overflowing-trash-bin-stockcake (2).jpg  
  inflating: dataset/Test/Garbage/overflowing-trash-bin-stockcake (3).jpg  
  inflating: dataset/Test/Garbage/overflowing-trash-bin-stockcake.jpg  
  inflating: dataset/Te

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gradio as gr

In [4]:
dataset_path ="/content/dataset"
train_dir = os.path.join(dataset_path, "Train")
validation_dir = os.path.join(dataset_path, "Validate")
test_dir = os.path.join(dataset_path, "Test")

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize the images to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),  # InceptionV3 input size
    batch_size=32,
    class_mode='binary'  # 'binary' for 2 classes (Garbage vs Streetdog)
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

Found 44 images belonging to 2 classes.
Found 33 images belonging to 2 classes.
Found 31 images belonging to 2 classes.


In [7]:
base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Add custom layers for binary classification (Garbage or Streetdog)
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Define the model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of InceptionV3 and train only the custom layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Save the trained model for later use


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 26s 26s/step - accuracy: 0.5000 - loss: 0.8308 - val_accuracy: 0.3438 - val_loss: 3.5075
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.5625 - loss: 2.4357 - val_accuracy: 1.0000 - val_loss: 0.1748
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 4/10


/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step - accuracy: 1.0000 - loss: 0.1058 - val_accuracy: 0.7812 - val_loss: 0.3276
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.7188 - loss: 0.5850 - val_accuracy: 1.0000 - val_loss: 0.0022
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step - accuracy: 0.9375 - loss: 0.1256 - val_accuracy: 0.9688 - val_loss: 0.1057
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.9167 - loss: 0.0974 - val_accuracy: 1.0000 - val_loss: 0.0079
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 38s 38s/step - accuracy: 1.0000 - loss: 0.0537 - val_accuracy: 0.8750 - val_loss: 0.1991


In [9]:
model.save("streetdog_model.h5")



In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", 'wb') as f:
  f.write(tflite_model)

Saved artifact at '/tmp/tmpei92ngiq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136138231991376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136138231991760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136138231993104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136138231993872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136138231993296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136138231994256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136138231992336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136138231992144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136138231994832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136138231995216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13613823199291

In [11]:
import gradio as gr
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the TFLite model using the `tf.lite.Interpreter`
interpreter = tf.lite.Interpreter(model_path="/content/model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Define the class labels (adjust as necessary)
class_labels = ['Garbage', 'Streetdog']  # Binary classification: 0 -> Garbage, 1 -> Streetdog

# Function to process the uploaded image and get predictions
def predict_image(img):
    # Preprocess the image to fit the model's input size and scale
    img = img.resize((299, 299))  # InceptionV3 input size
    img_array = np.array(img) / 255.0  # Normalize image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], img_array.astype(np.float32))

    # Run inference
    interpreter.invoke()

    # Get the output tensor
    prediction = interpreter.get_tensor(output_details[0]['index'])

    # Check prediction and provide labels accordingly
    if prediction[0] > 0.8:  # "Streetdog"
        label_text = "Streetdog"
    else:  # "Garbage"
        label_text = "Garbage"

    return label_text

# Create a Gradio interface
interface = gr.Interface(
    fn=predict_image,  # Function to get predictions
    inputs=gr.Image(type="pil"),  # Input: Image upload
    outputs=gr.Textbox(),  # Output: Textbox for label text with prediction and confidence
    live=True,  # Make prediction on live image update
    title="Streetdog vs Garbage Classifier",  # Interface title
    description="Upload an image to predict if it is a Streetdog or Garbage"
)

# Launch the Gradio app
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://231adefa663056421a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
